# Regresión Logística V. Decision Tree

In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score , cohen_kappa_score, roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_estand = pd.read_csv("airlines_estand.csv", index_col = 0)
df_estand.head(1)

,satisfaction,gender,customer_type,age,type_of_travel,class,flight_distance,seat_comfort,time_convenient,food_and_drink,...,entertainment,online_support,online_booking,on-board_service,leg_room_service,baggage_handling,checkin_service,cleanliness,online_boarding,departure_delay
0,0,0,0,1.041667,1,2,-1.400844,0,0,0,...,1,2,2,2,0,2,1,2,2,0.0


#### 1. Modelo con los datos estandarizados

##### 1.1. Primer modelo

Hasta ahora hemos ajustado el modelo usando una Regresión Logística, pero como hemos aprendido, podemos usar el Decision Tree en este tipo de problemas. Los objetivos de este pair programming:

1. Ajustad un modelo de Decision Tree a nuestros datos.

In [ ]:
# Separamos la variable respuesta de las variables predictoras.

X1 = df_estand.drop("satisfaction", axis = 1)
y1 = df_estand["satisfaction"]

In [ ]:
# Separamos en train y test para comprobar que nuestro modelo haga correctamente las predicciones.
x_train1, x_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size = 0.2, random_state = 42)

In [ ]:
# Instanciamos el modelo.

arbol1 = DecisionTreeClassifier(random_state =0)

# Ajustamos el modelo.

arbol1.fit(x_train1, y_train1)

DecisionTreeClassifier(random_state=0)

In [ ]:
"""# Sacamos la estructura de nuestro árbol.

fig = plt.figure(figsize = (10,6))
tree.plot_tree(arbol1, feature_names = x_train1.columns.tolist(), filled = True)
# plt.show()"""

'# Sacamos la estructura de nuestro árbol.\n\nfig = plt.figure(figsize = (10,6))\ntree.plot_tree(arbol1, feature_names = x_train1.columns.tolist(), filled = True)\n# plt.show()'

In [ ]:
# Hacemos las predicciones.
y_pred_test_estand = arbol1.predict(x_test1)
y_pred_train_estand = arbol1.predict(x_train1)

2. Calculad las métricas a nuestro nuevo modelo.

In [13]:
def metricas(clases_reales_test, clases_predichas_test, clases_reales_train, clases_predichas_train, modelo):
    """
    Esta función nos saca las métricas de nuestro modelo a predecir.
    Args:
    clases_reales_test(serie): los datos que tenemos.
    clases_predichas_test(serie): los datos que ha predicho.
    clases_reales_train(serie): los datos que tenemos.
    clases_predichas_train(serie): los datos que ha predicho.
    modelo(string): nombre que le damos a nuestro modelo.
    Returns: devuelve un dataframe con las métricas.
    """
    # Para el test
    accuracy_test = accuracy_score(clases_reales_test, clases_predichas_test)
    precision_test = precision_score(clases_reales_test, clases_predichas_test)
    recall_test = recall_score(clases_reales_test, clases_predichas_test)
    f1_test = f1_score(clases_reales_test, clases_predichas_test)
    kappa_test = cohen_kappa_score(clases_reales_test, clases_predichas_test)

    # Para el train
    accuracy_train = accuracy_score(clases_reales_train, clases_predichas_train)
    precision_train = precision_score(clases_reales_train, clases_predichas_train)
    recall_train = recall_score(clases_reales_train, clases_predichas_train)
    f1_train = f1_score(clases_reales_train, clases_predichas_train)
    kappa_train = cohen_kappa_score(clases_reales_train, clases_predichas_train)
    df = pd.DataFrame({"accuracy": [accuracy_test, accuracy_train],
                       "precision": [precision_test, precision_train],
                       "recall": [recall_test, recall_train],
                       "f1": [f1_test, f1_train],
                       "kapppa": [kappa_test, kappa_train],
                       "set": ["test", "train"]})
    df["modelo"] = modelo
    return df

In [ ]:
# sacamos las métricas para ver si hay overfitting o underfitting, para modificar la profundidad en función de estos resultados

dt_results1 = metricas(y_test1, y_pred_test_estand, y_train1, y_pred_train_estand, "Decission Tree Estandarizado I")
dt_results1

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.927164,0.92034,0.917345,0.91884,0.852779,test,Decission Tree Estandarizado I
1,1.000000,1.00000,1.000000,1.00000,1.000000,train,Decission Tree Estandarizado I


- Interpretamos los resultados:

En `accuracy` nuestros modelos aciertan un 0.92% en test y un 1% en train.

En `precision` nuestros modelos aciertan un 0.92% en test y un 1% en train a la hora de predecir positivos, es decir, en los pasajeros que no están satisfechos.

En `recall` nuestros modelos aciertan un 0.91% en test y un 1% en train, por lo que es capaz de identificar un 91% de los pasajeros no satisfechos o su totalidad.

En `kappa` nuestros modelos tienen un valor de 0.85 en test y 1% en train, por lo que nuestros modelos no están acertando en las predicciones por azar.

No obstante, es un claro caso de overfitting, porque las métricas de train son todas de 1%, lo cual indica que se está aprendiendo los datos de memoria.
**Vamos a intentar mejorar los resultados**

##### 1.2. Segundo modelo

In [ ]:
# Sacamos la raíz cuadrada del número de variables predictoras para poder establecer los hiperpámetros.
max_features = np.sqrt(len(x_train1.columns))
max_features

4.58257569495584

In [ ]:
# Sacamos la máxima profundidad a la que puede llegar nuestro árbol.
print(arbol1.tree_.max_depth)

35


In [ ]:
param = {"max_depth": [11, 13, 15],
         "max_features": [3,4, 5],
        "min_samples_split": [10, 50, 100, 300],
        "min_samples_leaf": [10,50, 100, 150]}

In [ ]:
# una vez creado el diccionario iniciaremos el modelo con GridSearch

gs = GridSearchCV(
            estimator=DecisionTreeClassifier(random_state= 42), # tipo de modelo que queremos hacer
            param_grid= param, # los hiperparámetros que debe usar
            cv=10, # crossvalidation
            verbose= 0)

In [ ]:
gs.fit(x_train1, y_train1)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [11, 13, 15], 'max_features': [3, 4, 5],
                         'min_samples_leaf': [10, 50, 100, 150],
                         'min_samples_split': [10, 50, 100, 300]})

In [ ]:
mejor_modelo1 = gs.best_estimator_
mejor_modelo1

DecisionTreeClassifier(max_depth=13, max_features=5, min_samples_leaf=10,
                       min_samples_split=50, random_state=42)

In [ ]:
y_pred_test_estand2 = mejor_modelo1.predict(x_test1)
y_pred_train_estand2 = mejor_modelo1.predict(x_train1)

In [ ]:
dt_results2 = metricas(y_test1, y_pred_test_estand2, y_train1, y_pred_train_estand2, "Decision tree Estandarizado II")
dt_results2

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.927241,0.920137,0.917773,0.918954,0.852944,test,Decision tree Estandarizado II
1,0.932563,0.927431,0.923554,0.925488,0.863899,train,Decision tree Estandarizado II


In [ ]:
df_decision_results = pd.concat([dt_results1, dt_results2], axis = 0)
df_decision_results

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.927164,0.920340,0.917345,0.918840,0.852779,test,Decission Tree Estandarizado I
1,1.000000,1.000000,1.000000,1.000000,1.000000,train,Decission Tree Estandarizado I
0,0.927241,0.920137,0.917773,0.918954,0.852944,test,Decision tree Estandarizado II
1,0.932563,0.927431,0.923554,0.925488,0.863899,train,Decision tree Estandarizado II


Como podemos ver, en este caso ya no hay overfitting, porque las métricas de train son menores de 1%, y son muy parecidas a las de test.

*Investigar las métricas en función de lo queremos*.

### 2. Modelo con los datos sin estandarizar

In [2]:
df_no_estand = pd.read_csv("../data/airlines_no_estand.csv", index_col = 0)
df_no_estand.head(1)

,satisfaction,gender,customer_type,age,type_of_travel,class,flight_distance,seat_comfort,time_convenient,food_and_drink,...,entertainment,online_support,online_booking,on-board_service,leg_room_service,baggage_handling,checkin_service,cleanliness,online_boarding,departure_delay
0,0,0,0,65,1,2,265,0,0,0,...,1,2,2,2,0,2,1,2,2,0


##### 2.1. Primer modelo

In [3]:
# Separamos la variable respuesta de las variables predictoras.

X3 = df_no_estand.drop("satisfaction", axis = 1)
y3 = df_no_estand["satisfaction"]

In [4]:
# Separamos en train y test para comprobar que nuestro modelo haga correctamente las predicciones.
x_train3, x_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size = 0.2, random_state = 42)

In [ ]:
# Instanciamos el modelo.

arbol3 = DecisionTreeClassifier(random_state =0)

# Ajustamos el modelo.

arbol3.fit(x_train3, y_train3)

DecisionTreeClassifier(random_state=0)

In [ ]:
# Hacemos las predicciones.
y_pred_test_no_estand1 = arbol3.predict(x_test3)
y_pred_train_no_estand1 = arbol3.predict(x_train3)

In [ ]:
# sacamos las métricas para ver si hay overfitting o underfitting, para modificar la profundidad en función de estos resultados

dt_results3 = metricas(y_test3, y_pred_test_no_estand1, y_train3, y_pred_train_no_estand1, "Decission Tree No Estandarizado I")
dt_results3

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.926894,0.920003,0.917088,0.918543,0.852235,test,Decission Tree No Estandarizado I
1,1.000000,1.000000,1.000000,1.000000,1.000000,train,Decission Tree No Estandarizado I


- Interpretamos los resultados:

En `accuracy` nuestros modelos aciertan un 0.92% en test y un 1% en train.

En `precision` nuestros modelos aciertan un 0.92% en test y un 1% en train a la hora de predecir positivos, es decir, en los pasajeros que no están satisfechos.

En `recall` nuestros modelos aciertan un 0.91% en test y un 1% en train, por lo que es capaz de identificar un 91% de los pasajeros no satisfechos o su totalidad.

En `kappa` nuestros modelos tienen un valor de 0.85 en test y 1% en train, por lo que nuestros modelos no están acertando en las predicciones por azar.

No obstante, es un claro caso de overfitting, porque las métricas de train son todas de 1%, lo cual indica que se está aprendiendo los datos de memoria.
**Vamos a intentar mejorar los resultados**

##### 2.2. Segundo modelo.

In [ ]:

gs2 = GridSearchCV(
            estimator=DecisionTreeClassifier(random_state= 42), # tipo de modelo que queremos hacer
            param_grid= param, # los hiperparámetros que debe usar
            cv=10, # crossvalidation
            verbose= 0)

In [ ]:
gs2.fit(x_train3, y_train3)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [11, 13, 15], 'max_features': [3, 4, 5],
                         'min_samples_leaf': [10, 50, 100, 150],
                         'min_samples_split': [10, 50, 100, 300]})

In [ ]:
mejor_modelo2 = gs2.best_estimator_
mejor_modelo2

DecisionTreeClassifier(max_depth=13, max_features=5, min_samples_leaf=10,
                       min_samples_split=50, random_state=42)

In [ ]:
y_pred_test_no_estand2 = mejor_modelo1.predict(x_test3)
y_pred_train_no_estand2 = mejor_modelo1.predict(x_train3)

In [ ]:
dt_results4 = metricas(y_test3, y_pred_test_no_estand2, y_train3, y_pred_train_no_estand2, "Decision tree No Estandarizado II")
dt_results4

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.896481,0.845935,0.941071,0.89097,0.792963,test,Decision tree No Estandarizado II
1,0.897579,0.848080,0.943079,0.89306,0.795320,train,Decision tree No Estandarizado II


In [ ]:
df_decision_results2 = pd.concat([dt_results3, dt_results4], axis = 0)
df_decision_results2

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.926894,0.920003,0.917088,0.918543,0.852235,test,Decission Tree No Estandarizado I
1,1.000000,1.000000,1.000000,1.000000,1.000000,train,Decission Tree No Estandarizado I
0,0.896481,0.845935,0.941071,0.890970,0.792963,test,Decision tree No Estandarizado II
1,0.897579,0.848080,0.943079,0.893060,0.795320,train,Decision tree No Estandarizado II


Como podemos ver, en este caso ya no hay overfitting, porque las métricas de train son menores de 1%, y son muy parecidas a las de test.

*Investigar las métricas en función de lo queremos*.

In [ ]:
df_decision_final = pd.concat([df_decision_results, df_decision_results2], axis = 0)
df_decision_final

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.927164,0.920340,0.917345,0.918840,0.852779,test,Decission Tree Estandarizado I
1,1.000000,1.000000,1.000000,1.000000,1.000000,train,Decission Tree Estandarizado I
0,0.927241,0.920137,0.917773,0.918954,0.852944,test,Decision tree Estandarizado II
1,0.932563,0.927431,0.923554,0.925488,0.863899,train,Decision tree Estandarizado II
0,0.926894,0.920003,0.917088,0.918543,0.852235,test,Decission Tree No Estandarizado I
1,1.000000,1.000000,1.000000,1.000000,1.000000,train,Decission Tree No Estandarizado I
0,0.896481,0.845935,0.941071,0.890970,0.792963,test,Decision tree No Estandarizado II
1,0.897579,0.848080,0.943079,0.893060,0.795320,train,Decision tree No Estandarizado II


Nuestras mejores métricas son en nuestros segundos modelos. Por tanto, vamos a eliminar de los resultados finales aquellos que no nos sirven.

In [ ]:
df_dt = df_decision_final.reset_index(drop = True)
df_dt.drop([0, 1, 4, 5], axis = 0, inplace = True)
df_dt

,accuracy,precision,recall,f1,kapppa,set,modelo
2,0.927241,0.920137,0.917773,0.918954,0.852944,test,Decision tree Estandarizado II
3,0.932563,0.927431,0.923554,0.925488,0.863899,train,Decision tree Estandarizado II
6,0.896481,0.845935,0.941071,0.890970,0.792963,test,Decision tree No Estandarizado II
7,0.897579,0.848080,0.943079,0.893060,0.795320,train,Decision tree No Estandarizado II


Recordamos que nuestra variable respuesta es:

0 = Sastifecho
1 = No satisfecho.

Consideramos más importante los **falsos positivos**, que en este caso es predecir que el cliente no está satisfecho cuando realmente sí que lo está. Teniendo esto en cuenta, aparte de la métrica de kappa, que es la más importante, tenemos que priorizar la métrica de *recall*, ya que preferimos equivarnos en los falsos positivos, es decir, preferimos predecir que hay más gente que no está satisfecha y que realmente sí que lo están.

Se puede apreciar que nuestra métrica de recall es muy buena, por lo que nuestro modelo está capturando entre el 0.89 y  0.91-0.92% de los casos negativos.

In [ ]:
df_rlog = pd.read_csv("airlines_metricas.csv", index_col = 0)
df_rlog

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.882892,0.876625,0.860557,0.868517,0.762967,test,Regresión logistica
1,0.880592,0.873840,0.860987,0.867366,0.758796,train,Regresión logistica
0,0.883084,0.875958,0.861927,0.868886,0.763406,test,Regresión logistica II
1,0.880168,0.872739,0.861348,0.867006,0.757974,train,Regresión logistica II


In [ ]:
df_decision_final = pd.concat([df_rlog, df_dt], axis = 0)
df_decision_final

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.882892,0.876625,0.860557,0.868517,0.762967,test,Regresión logistica
1,0.880592,0.873840,0.860987,0.867366,0.758796,train,Regresión logistica
0,0.883084,0.875958,0.861927,0.868886,0.763406,test,Regresión logistica II
1,0.880168,0.872739,0.861348,0.867006,0.757974,train,Regresión logistica II
2,0.927241,0.920137,0.917773,0.918954,0.852944,test,Decision tree Estandarizado II
3,0.932563,0.927431,0.923554,0.925488,0.863899,train,Decision tree Estandarizado II
6,0.896481,0.845935,0.941071,0.890970,0.792963,test,Decision tree No Estandarizado II
7,0.897579,0.848080,0.943079,0.893060,0.795320,train,Decision tree No Estandarizado II


Además, podemos ver que nuestros modelos realizados con *Decission Tree* son mejores que las realizadas por Regresión Lógistica.


# Regresión logística VI. Random Forest.

#### 1. Modelo con los datos estandarizados.

1. Ajustad un modelo de Random Forest a nuestros datos.


Ya tenemos los datos separados en X e y. Además, también hemos visto que nuestro modelo funciona mejor cuando ajustamos los parámetros, ya que de lo contrario hace *overfitting*.

In [5]:
# Establecemos los parámetros para el random forest.
param2 = {"max_depth": [11, 13], # Lo dejamos como máximo en 13 porque es el parámetro que ha cogido previamente con el Decision Tree.
        "max_features": [5, 6], # En Decision Tree coge 5, así que lo subimos también a 6.
        # Teniendo en cuenta nuestros datos y el mejor modelo del Decision Tree, estos parámetros los dejamos así:
        "min_samples_split": [200, 250, 300],
        "min_samples_leaf": [100, 125, 150]}

In [ ]:
# ajustamos el GridSearch para buscar el mejor modelo para nuestros datos.

gs_rf = GridSearchCV(
            estimator=RandomForestClassifier(random_state=42), # tipo de modelo que queremos hacer
            param_grid= param2, # que hiperparámetros queremos que tenga. Están definidos arriba.
            cv=10, # crossvalidation
            verbose= 0)

In [ ]:
# Ajustamos el modelo que hemos definido en el GridSearch a nuestros datos.

gs_rf.fit(x_train1, y_train1)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [11, 13], 'max_features': [5, 6],
                         'min_samples_leaf': [100, 125, 150],
                         'min_samples_split': [200, 250, 300]})

In [ ]:
# Sacamos el mejor modelo.

bosque = gs_rf2.best_estimator_
bosque

RandomForestClassifier(max_depth=11, max_features=6, min_samples_leaf=100,
                       min_samples_split=200, random_state=42)

In [ ]:
# Realizamos las predicciones.

y_pred_test_rf = bosque.predict(x_test1)
y_pred_train_rf = bosque.predict(x_train1)

2. Calculad las métricas a nuestro nuevo modelo.

In [ ]:
# Obtenemos los resultados de nuestro modelo.

dt_results = metricas(y_test1, y_pred_test_rf, y_train1, y_pred_train_rf, "Random Forest")
dt_results

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.924084,0.918702,0.911777,0.915227,0.846494,test,Random Forest
1,0.922640,0.916312,0.912772,0.914538,0.843878,train,Random Forest


En este caso, tampoco tenemos *overfitting*, ya que los resultados de test y train son bastante similares. Kappa está 0.84 para ambos, mientras que *accuracy* está en 0.92 y el resto están en 0.91. Por tanto, son métricas bastante buenas.

Recordamos que nuestra variable respuesta es:

0 = Sastifecho
1 = No satisfecho.

Consideramos más importante los **falsos positivos**, que en este caso es predecir que el cliente no está satisfecho cuando realmente sí que lo está. Teniendo esto en cuenta, aparte de la métrica de kappa, que es la más importante, tenemos que priorizar la métrica de *recall*, ya que preferimos equivarnos en los falsos positivos, es decir, preferimos predecir que hay más gente que no está satisfecha y que realmente sí que lo están.

Se puede apreciar que nuestra métrica de recall es muy buena, por lo que nuestro modelo está capturando entre el 0.89 y  0.91-0.92% de los casos negativos.

#### 2. Modelo con los datos sin estandarizar

In [7]:
# Utilizamos el GridSearch para buscar el mejor modelo para nuestros datos. Los parámetros ya están establecidos previamente.
gs_rf2 = GridSearchCV(
            estimator=RandomForestClassifier(random_state=42), # tipo de modelo que queremos hacer
            param_grid= param2, # que hiperparámetros queremos que tenga. Están definidos arriba.
            cv=10, # crossvalidation
            verbose= 0)

In [9]:
# Ajustamos el modelo que hemos definido en el GridSearch a nuestros datos.

gs_rf2.fit(x_train3, y_train3)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [11, 13], 'max_features': [5, 6],
                         'min_samples_leaf': [100, 125, 150],
                         'min_samples_split': [200, 250, 300]})

In [10]:
# Sacamos el mejor modelo.

bosque2 = gs_rf2.best_estimator_
bosque2

RandomForestClassifier(max_depth=11, max_features=6, min_samples_leaf=100,
                       min_samples_split=200, random_state=42)

In [11]:
# Realizamos las predicciones.

y_pred_test_rf2 = bosque2.predict(x_test3)
y_pred_train_rf2 = bosque2.predict(x_train3)

In [14]:
dt_results2 = metricas(y_test3, y_pred_test_rf2, y_train3, y_pred_train_rf2, "Random Forest No Estandarizado")
dt_results2

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.924084,0.918702,0.911777,0.915227,0.846494,test,Random Forest No Estandarizado
1,0.922640,0.916312,0.912772,0.914538,0.843878,train,Random Forest No Estandarizado


De nuevo, tenemos buenos resultados en todas nuestras métricas.

3. Comparad las métricas con los modelos hechos hasta ahora. ¿Cuál es mejor?

In [21]:
df_decision_final = pd.concat([df_decision_final, dt_results, dt_results2], axis = 0)
df_decision_final

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.882892,0.876625,0.860557,0.868517,0.762967,test,Regresión logistica
1,0.880592,0.873840,0.860987,0.867366,0.758796,train,Regresión logistica
0,0.883084,0.875958,0.861927,0.868886,0.763406,test,Regresión logistica II
1,0.880168,0.872739,0.861348,0.867006,0.757974,train,Regresión logistica II
2,0.927241,0.920137,0.917773,0.918954,0.852944,test,Decision tree Estandarizado II
3,0.932563,0.927431,0.923554,0.925488,0.863899,train,Decision tree Estandarizado II
6,0.896481,0.845935,0.941071,0.890970,0.792963,test,Decision tree No Estandarizado II
7,0.897579,0.848080,0.943079,0.893060,0.795320,train,Decision tree No Estandarizado II
0,0.924084,0.918702,0.911777,0.915227,0.846494,test,Random Forest
1,0.922640,0.916312,0.912772,0.914538,0.843878,train,Random Forest


#### Conclusiones finales:

En conclusión, parece que nuestro mejor modelo ha sido el **Decision Tree** realizado con los datos estandarizados, aunque el **Random Forest** con los datos estandarizados y sin estandarizar también dan buenos resultados. Estos dos modelos son los que tienen las métricas más altas, lo cual es positivo. No obstante, podríamos mejorar las métricas realizando otros modelos, por ejemplo, quitando o gestionando los outliers o modificando el encoding.

In [22]:
df_decision_final.to_csv("../data/metricas_finales.csv")